# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_weather = pd.read_csv("Cities_Weather_Data.csv")
cities_weather.head()

,Unnamed: 0,city,Lat,Lng,temp,hum,pres,cloud,wind,country,date
0,0,manggar,-2.870315,108.279273,80.24,80.0,1008.0,98.0,13.06,ID,1.595890e+09
1,1,dakoro,14.508918,6.765460,82.20,62.0,1011.0,70.0,13.24,NE,1.595827e+09
2,2,azovo,54.704767,73.041044,60.80,63.0,1009.0,4.0,4.47,RU,1.595891e+09
3,3,san juan,38.665911,-121.285525,91.56,27.0,1010.0,1.0,6.93,US,1.595855e+09
4,4,husavik,66.044971,-17.338344,50.00,61.0,1012.0,40.0,14.99,IS,1.595821e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [13]:
locations = cities_weather[["Lat", "Lng"]].astype(float)
humidity = cities_weather["hum"].astype(float)
fig = gmaps.figure()
# heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=100, point_radius = 8)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
vacation_cities = cities_weather[cities_weather['temp']>=70] 
vacation_cities = cities_weather[cities_weather['temp']<=80]
vacation_cities = cities_weather[cities_weather['hum']<=70]
vacation_cities = cities_weather[cities_weather['wind']<=8]
vacation_cities = cities_weather[cities_weather['cloud']<=5]

vacation_cities

,Unnamed: 0,city,Lat,Lng,temp,hum,pres,cloud,wind,country,date
2,2,azovo,54.704767,73.041044,60.80,63.0,1009.0,4.0,4.47,RU,1.595891e+09
3,3,san juan,38.665911,-121.285525,91.56,27.0,1010.0,1.0,6.93,US,1.595855e+09
8,8,port alfred,-33.586407,26.885145,57.11,75.0,1025.0,0.0,15.84,ZA,1.595826e+09
10,10,cape town,-33.924869,18.424055,53.51,93.0,1020.0,0.0,1.12,ZA,1.595829e+09
15,15,zhangye,38.925875,100.449818,71.13,28.0,1009.0,0.0,6.42,CN,1.595888e+09
...,...,...,...,...,...,...,...,...,...,...,...
583,592,baiao,41.161772,-8.035514,66.70,84.0,1018.0,0.0,1.99,PT,1.595827e+09
589,598,aleksinac,43.540990,21.718370,73.40,69.0,1017.0,0.0,1.12,RS,1.595820e+09
590,599,xichang,27.894504,102.264449,63.10,73.0,1011.0,3.0,1.88,CN,1.595889e+09
592,601,diego de almagro,-26.390249,-70.047530,80.26,16.0,1018.0,0.0,10.20,CL,1.595849e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
hotel_df = vacation_cities
# hotel_df.reset_index(inplace=True)
hotel_df

,index,Unnamed: 0,city,Lat,Lng,temp,hum,pres,cloud,wind,country,date,Hotel_Name
0,2,2,azovo,54.704767,73.041044,60.80,63.0,1009.0,4.0,4.47,RU,1.595891e+09,Azovo
1,3,3,san juan,38.665911,-121.285525,91.56,27.0,1010.0,1.0,6.93,US,1.595855e+09,Citrus Heights
2,8,8,port alfred,-33.586407,26.885145,57.11,75.0,1025.0,0.0,15.84,ZA,1.595826e+09,Port Alfred
3,10,10,cape town,-33.924869,18.424055,53.51,93.0,1020.0,0.0,1.12,ZA,1.595829e+09,Cape Town
4,15,15,zhangye,38.925875,100.449818,71.13,28.0,1009.0,0.0,6.42,CN,1.595888e+09,Zhangye
...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,583,592,baiao,41.161772,-8.035514,66.70,84.0,1018.0,0.0,1.99,PT,1.595827e+09,Campelo
169,589,598,aleksinac,43.540990,21.718370,73.40,69.0,1017.0,0.0,1.12,RS,1.595820e+09,Aleksinac
170,590,599,xichang,27.894504,102.264449,63.10,73.0,1011.0,3.0,1.88,CN,1.595889e+09,Liangshan Yi Autonomous Prefecture
171,592,601,diego de almagro,-26.390249,-70.047530,80.26,16.0,1018.0,0.0,10.20,CL,1.595849e+09,Diego de Almagro


In [38]:


params = {
    "key": g_key,
    "radius" : 5000
}

for index, row in hotel_df.iterrows():
    

    # base_url = "http://api.openweathermap.org/data/2.5/onecall/timemachine"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row['Lat']
    lon = row['Lng']
    
    params["location"] = f"{lat},{lon}"

    hotel = requests.get(base_url,params=params)

    hotel = hotel.json()
    
    try:
        hotel_df.loc[index, "Hotel_Name"] = hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

C:\Users\chris\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [43]:
hotel_df

,index,Unnamed: 0,city,Lat,Lng,temp,hum,pres,cloud,wind,country,date,Hotel_Name
0,2,2,azovo,54.704767,73.041044,60.80,63.0,1009.0,4.0,4.47,RU,1.595891e+09,Azovo
1,3,3,san juan,38.665911,-121.285525,91.56,27.0,1010.0,1.0,6.93,US,1.595855e+09,Citrus Heights
2,8,8,port alfred,-33.586407,26.885145,57.11,75.0,1025.0,0.0,15.84,ZA,1.595826e+09,Port Alfred
3,10,10,cape town,-33.924869,18.424055,53.51,93.0,1020.0,0.0,1.12,ZA,1.595829e+09,Cape Town
4,15,15,zhangye,38.925875,100.449818,71.13,28.0,1009.0,0.0,6.42,CN,1.595888e+09,Zhangye
...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,583,592,baiao,41.161772,-8.035514,66.70,84.0,1018.0,0.0,1.99,PT,1.595827e+09,Campelo
169,589,598,aleksinac,43.540990,21.718370,73.40,69.0,1017.0,0.0,1.12,RS,1.595820e+09,Aleksinac
170,590,599,xichang,27.894504,102.264449,63.10,73.0,1011.0,3.0,1.88,CN,1.595889e+09,Liangshan Yi Autonomous Prefecture
171,592,601,diego de almagro,-26.390249,-70.047530,80.26,16.0,1018.0,0.0,10.20,CL,1.595849e+09,Diego de Almagro


In [61]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Azovo</dd>\n<dt>City</dt><dd>azovo</dd>\n<dt>Country</dt><dd>RU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Citrus Heights</dd>\n<dt>City</dt><dd>san juan</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Port Alfred</dd>\n<dt>City</dt><dd>port alfred</dd>\n<dt>Country</dt><dd>ZA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Cape Town</dd>\n<dt>City</dt><dd>cape town</dd>\n<dt>Country</dt><dd>ZA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Zhangye</dd>\n<dt>City</dt><dd>zhangye</dd>\n<dt>Country</dt><dd>CN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Busselton</dd>\n<dt>City</dt><dd>busselton</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Mount Gambier</dd>\n<dt>City</dt><dd>mount gambier</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Tsiombe District</dd>\n<dt>City</dt><dd>tsihombe</dd>\n<dt>Country</dt><dd>MG</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Bredasdorp</dd>\n<dt>City</dt><dd>bredasdor

In [65]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))